In [0]:
import os
path = "/content/drive/Colab Notebooks/2048-api"
os.chdir(path)
os.listdir(path)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataLoader import MyDataset as MyDataset
from torchvision import transforms
from torch.autograd import Variable
import time

定义网络模型：RNN变体——LSTM

In [0]:
batch_size = 32000
start_time = time.time()

class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()

		self.RNN = nn.LSTM(
			input_size = 4,
			hidden_size = 300,
			num_layers = 4,
			batch_first=True)
		self.fc1 = nn.Linear(300, 64)
		self.fc2 = nn.Linear(64, 4)

	def forward(self, x):
		x, (h_n, h_c) = self.RNN(x, None)
		x = x[:, -1 ,:]
		x = self.fc1(x)
		x = self.fc2(x)
		return F.log_softmax(x, dim=1)

导入数据

In [0]:
def load_data():
	train_data = MyDataset(
		root = './Dataset/Train.csv',
		transform=transforms.Compose(
			[transforms.ToTensor()]))

	train = torch.utils.data.DataLoader(
		train_data,
		batch_size=batch_size,
		shuffle=True,
		num_workers=0)

	test_data = MyDataset(
		root = './Dataset/Train.csv',
		transform=transforms.Compose(
			[transforms.ToTensor()]))

	test = torch.utils.data.DataLoader(
		test_data,
		batch_size=batch_size,
		shuffle=True,
		num_workers=0)
	return train, test



训练函数

In [0]:
# Train the net
def train(model, epoch, train_loader, optimizer):
	model.train()

	for idx, (data, target) in enumerate(train_loader):
		
		data = data.type(torch.float)
		data = Variable(data.view(-1,4,4))

		if torch.cuda.is_available():
			data = Variable(data).cuda()
			target = Variable(target).cuda()
			model.cuda()

		output = model(data)

		optimizer.zero_grad()
		# target = target.repeat(12)
		loss = F.nll_loss(output, target)
		loss.backward()
		optimizer.step()
		
		if idx % 10 == 0:
			predict = output.data.max(1)[1]
			num = predict.eq(target.data).sum()
			correct = 100.0*num/batch_size
			t = time.time()-start_time
			# print("\t\t\t\t\t", predict[0:20])
			# print("\t\t\t\t\t", target[0:20])
			print('Train epoch: %d   Loss: %.3f    ' % (epoch+1, loss), \
				'Accuracy: %0.2f' % correct, '%', '\tTotal Time: %0.2f' % t)




主函数

In [0]:
def main():
	model = Net()

	train_loader, test_loader = load_data()

	# optimizer = optim.RMSprop(model.parameters(), lr=0.001)
	optimizer = optim.Adam(model.parameters(), lr=0.001)
	epochs = 20

	for epoch in range(epochs):
		if(epoch>10):
			optimizer = optim.Adam(model.parameters(), lr=0.0001)
		train(model, epoch, train_loader, optimizer)
		torch.save(model, 'model_'+str(epoch) + '.pth')

	torch.save(model, 'model.pth')
	print("Time used:", time.time()-start_time)



In [0]:
if __name__ == '__main__':
	main()

Train epoch: 1   Loss: 1.387     Accuracy: 24.00 % 	Total Time: 14.90
Train epoch: 1   Loss: 1.382     Accuracy: 37.00 % 	Total Time: 26.19
Train epoch: 1   Loss: 1.318     Accuracy: 37.00 % 	Total Time: 37.78
Train epoch: 1   Loss: 1.302     Accuracy: 37.00 % 	Total Time: 49.40
Train epoch: 1   Loss: 1.274     Accuracy: 36.00 % 	Total Time: 61.02
Train epoch: 1   Loss: 1.266     Accuracy: 37.00 % 	Total Time: 72.67
Train epoch: 1   Loss: 1.234     Accuracy: 37.00 % 	Total Time: 84.30
Train epoch: 1   Loss: 1.197     Accuracy: 37.00 % 	Total Time: 95.97
Train epoch: 1   Loss: 1.185     Accuracy: 38.00 % 	Total Time: 107.59
Train epoch: 1   Loss: 1.174     Accuracy: 38.00 % 	Total Time: 119.19
Train epoch: 1   Loss: 1.170     Accuracy: 39.00 % 	Total Time: 130.80
Train epoch: 1   Loss: 1.166     Accuracy: 39.00 % 	Total Time: 142.43
Train epoch: 1   Loss: 1.167     Accuracy: 38.00 % 	Total Time: 154.11
Train epoch: 1   Loss: 1.160     Accuracy: 39.00 % 	Total Time: 165.77
Train epoch: 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train epoch: 2   Loss: 1.158     Accuracy: 39.00 % 	Total Time: 223.33
Train epoch: 2   Loss: 1.156     Accuracy: 39.00 % 	Total Time: 235.11
Train epoch: 2   Loss: 1.153     Accuracy: 40.00 % 	Total Time: 246.73
Train epoch: 2   Loss: 1.151     Accuracy: 40.00 % 	Total Time: 258.36
Train epoch: 2   Loss: 1.149     Accuracy: 40.00 % 	Total Time: 270.02
Train epoch: 2   Loss: 1.142     Accuracy: 40.00 % 	Total Time: 281.66
Train epoch: 2   Loss: 1.137     Accuracy: 40.00 % 	Total Time: 293.31
Train epoch: 2   Loss: 1.139     Accuracy: 41.00 % 	Total Time: 304.95
Train epoch: 2   Loss: 1.139     Accuracy: 41.00 % 	Total Time: 316.58
Train epoch: 2   Loss: 1.132     Accuracy: 41.00 % 	Total Time: 328.24
Train epoch: 2   Loss: 1.129     Accuracy: 41.00 % 	Total Time: 339.89
Train epoch: 2   Loss: 1.131     Accuracy: 41.00 % 	Total Time: 351.53
Train epoch: 2   Loss: 1.128     Accuracy: 42.00 % 	Total Time: 363.17
Train epoch: 2   Loss: 1.126     Accuracy: 41.00 % 	Total Time: 374.80
Train 